In [1]:
import awkward as ak
from coffea.nanoevents import NanoEventsFactory, NanoAODSchema
import time
from coffea import processor, hist
from coffea.util import load, save
import json
import glob
import os
import argparse
import numpy as np
from coffea import lumi_tools
import numba
import pandas as pd

In [2]:
infile= "wza_UL18_sum.root" # ZZ
dataset="WZG"
year='2018'

In [3]:
# Trigger set
triggers = {
    'Egamma':{
    "2018":["Ele23_Ele12_CaloIdL_TrackIdL_IsoVL","Ele32_WPTight_Gsf"]
    },
    'SingleMuon':{
    "2018":["IsoMu24"]
    },
    'DoubleMuon':{
    "2018":["Mu17_TrkIsoVVL_Mu8_TrkIsoVVL_DZ_Mass3p8"]
    },
    'MuonEG':{
    "2018":["Mu8_TrkIsoVVL_Ele23_CaloIdL_TrackIdL_IsoVL_DZ","Mu23_TrkIsoVVL_Ele12_CaloIdL_TrackIdL_IsoVL"]
    },
}

events =  NanoEventsFactory.from_root(infile, schemaclass=NanoAODSchema).events()

# Data or MC
isData = "genWeight" not in events.fields


def sort_by_pt(ele, pho, jet,muon):
    ele  = ele[ak.argsort(ele.pt, ascending=False, axis=1)]
    pho  = pho[ak.argsort(pho.pt, ascending=False, axis=1)]
    jet  = jet[ak.argsort(jet.pt, ascending=False, axis=1)]
    muon = muon[ak.argsort(muon.pt, ascending=False, axis=1)] 
    return ele, pho, jet, muon

from coffea.nanoevents.methods import vector
ak.behavior.update(vector.behavior)

def TLorentz_vector(vec):
    vec = ak.zip(
        {"x": vec.x, "y": vec.y, "z": vec.z, "t": vec.t},
        with_name="LorentzVector",
    )
    return vec

def TLorentz_vector_cylinder(vec):

    vec = ak.zip(
        {
            "pt": vec.pt,
            "eta": vec.eta,
            "phi": vec.phi,
            "mass": vec.mass,
        },
        with_name="PtEtaPhiMLorentzVector",
    )

    return vec

### Trigger

In [4]:
Initial_events = events
# Good Run ( Golden Json files )
from coffea import lumi_tools


triggers_mask = np.zeros(len(events),dtype=np.bool)
for key in triggers.keys():
    for t in triggers[key][year]:
        triggers_mask = triggers_mask | events.HLT[t]

events.Electron, events.Photon, events.Jet, events.Muon = sort_by_pt(events.Electron, events.Photon, events.Jet, events.Muon)

# Apply cut1: Triggers
events = events[triggers_mask]

cut1 = np.ones(len(events))

Electron = events.Electron
LooseElectron = events.Electron
LoosebutNotTightElectron = events.Electron

Muon = events.Muon
LooseMuon = events.Muon
LoosebutNotTightMuon = events.Muon


Photon = events.Photon
MET = events.MET
Jet = events.Jet
print("passing skimming & Triggers ",len(cut1))

passing skimming & Triggers  89755


### Lepton

In [5]:
# Muon selection
MuSelmask = (
    (Muon.pt >= 10)
    & (abs(Muon.eta) <= 2.5)
    & (Muon.tightId)
    & (Muon.pfRelIso04_all < 0.15)
    & (Muon.genPartFlav == 1)
)
Muon = Muon[MuSelmask]

# Loose Muon selection
MuSelmask = (
    (LooseMuon.pt >= 10)
    & (abs(LooseMuon.eta) <= 2.5)
    & (LooseMuon.looseId)
    & (LooseMuon.pfRelIso04_all < 0.4)
)
LooseMuon = LooseMuon[MuSelmask]

# Loose but not tight Muon selection
MuSelmask = (
    (LoosebutNotTightMuon.pt >= 10)
    & (abs(LoosebutNotTightMuon.eta) <= 2.5)
    & (LoosebutNotTightMuon.looseId)
    & (LoosebutNotTightMuon.pfRelIso04_all < 0.4)
    & (LoosebutNotTightMuon.pfRelIso04_all >= 0.15)
)
LoosebutNotTightMuon = LoosebutNotTightMuon[MuSelmask]

# Electron selection
EleSelmask = (
    (Electron.pt >= 10)
    & (np.abs(Electron.eta + Electron.deltaEtaSC) < 1.479)
    & (Electron.cutBased > 2)
    & (abs(Electron.dxy) < 0.05)
    & (abs(Electron.dz) < 0.1)
    & (Electron.genPartFlav == 1)
) | (
    (Electron.pt >= 10)
    & (np.abs(Electron.eta + Electron.deltaEtaSC) > 1.479)
    & (np.abs(Electron.eta + Electron.deltaEtaSC) <= 2.5)
    & (Electron.cutBased > 2)
    & (abs(Electron.dxy) < 0.1)
    & (abs(Electron.dz) < 0.2)
    & (Electron.genPartFlav == 1)
)
Electron = Electron[EleSelmask]


# Loose Electron selection
EleSelmask = (
    (LooseElectron.pt >= 10)
    & (np.abs(LooseElectron.eta + LooseElectron.deltaEtaSC) < 1.479)
    & (LooseElectron.cutBased >= 1)
    & (abs(LooseElectron.dxy) < 0.05)
    & (abs(LooseElectron.dz) < 0.1)
    & (LooseElectron.genPartFlav == 1)
) | (
    (LooseElectron.pt >= 10)
    & (np.abs(LooseElectron.eta + LooseElectron.deltaEtaSC) > 1.479)
    & (np.abs(LooseElectron.eta + LooseElectron.deltaEtaSC) <= 2.5)
    & (LooseElectron.cutBased >= 1)
    & (abs(LooseElectron.dxy) < 0.1)
    & (abs(LooseElectron.dz) < 0.2)
    & (LooseElectron.genPartFlav == 1)
)
LooseElectron = LooseElectron[EleSelmask]

# Loose Electron selection
EleSelmask = (
    (LoosebutNotTightElectron.pt >= 10)
    & (np.abs(LoosebutNotTightElectron.eta + LoosebutNotTightElectron.deltaEtaSC) < 1.479)
    & (LoosebutNotTightElectron.cutBased >= 1)
    & (LoosebutNotTightElectron.cutBased < 3)
    & (abs(LoosebutNotTightElectron.dxy) < 0.05)
    & (abs(LoosebutNotTightElectron.dz) < 0.1)
    & (LoosebutNotTightElectron.genPartFlav == 1)
) | (
    (LoosebutNotTightElectron.pt >= 10)
    & (np.abs(LoosebutNotTightElectron.eta + LoosebutNotTightElectron.deltaEtaSC) > 1.479)
    & (np.abs(LoosebutNotTightElectron.eta + LoosebutNotTightElectron.deltaEtaSC) <= 2.5)
    & (LoosebutNotTightElectron.cutBased >= 1)
    & (LoosebutNotTightElectron.cutBased < 3)
    & (abs(LoosebutNotTightElectron.dxy) < 0.1)
    & (abs(LoosebutNotTightElectron.dz) < 0.2)
    & (LoosebutNotTightElectron.genPartFlav == 1)
)
LoosebutNotTightElectron = LoosebutNotTightElectron[EleSelmask]

---
---

# eee channel
### OSSF

In [6]:
# remove evt with conataining loose but not tight leptons

Loose_lepton_veto = ((ak.num(LoosebutNotTightElectron) + ak.num(LoosebutNotTightMuon)) == 0)
eee_ch_mask       = (ak.num(Electron)==3) & (ak.num(Muon)==0)

Ele_Mask = (Loose_lepton_veto & eee_ch_mask)

Electron = Electron[Ele_Mask]
LooseElectron = LooseElectron[Ele_Mask]
LoosebutNotTightElectron = LoosebutNotTightElectron[Ele_Mask]

Muon = Muon[Ele_Mask]
LooseMuon = LooseMuon[Ele_Mask]
LoosebutNotTightMuon = LoosebutNotTightMuon[Ele_Mask]

Photon = Photon[Ele_Mask]
MET = MET[Ele_Mask]
Jet = Jet[Ele_Mask]
events = events[Ele_Mask]

print("passing eee ch mask: ",len(events))


##----------- Cut flow4: OSSF
# OSSF index maker
@numba.njit
def find_3lep(events_leptons, builder):
    for leptons in events_leptons:

        builder.begin_list()
        nlep = len(leptons)
        for i0 in range(nlep):
            for i1 in range(i0 + 1, nlep):
                if leptons[i0].charge + leptons[i1].charge != 0:
                    continue

                for i2 in range(nlep):
                    if len({i0, i1, i2}) < 3:
                        continue
                    builder.begin_tuple(3)
                    builder.index(0).integer(i0)
                    builder.index(1).integer(i1)
                    builder.index(2).integer(i2)
                    builder.end_tuple()
        builder.end_list()
    return builder

eee_triplet_idx = find_3lep(Electron, ak.ArrayBuilder()).snapshot()
ossf_mask = ak.num(eee_triplet_idx) == 2

for i,j in zip(Electron.charge[:5],eee_triplet_idx[:5]):
    print(i,j)

eee_triplet_idx = eee_triplet_idx[ossf_mask]
Electron = Electron[ossf_mask]
Photon = Photon[ossf_mask]
Jet = Jet[ossf_mask]
MET = MET[ossf_mask]
events = events[ossf_mask]
print("passing ossf: ",len(events))

Triple_electron = [Electron[eee_triplet_idx[idx]] for idx in "012"]
Triple_eee = ak.zip(
    {
        "lep1": Triple_electron[0],
        "lep2": Triple_electron[1],
        "lep3": Triple_electron[2],
        "p4": TLorentz_vector(Triple_electron[0] + Triple_electron[1]),
    }
)

print(Triple_eee.lep1.pt)
print(Triple_eee.p4.mass)
bestZ_idx = ak.singletons(ak.argmin(abs(Triple_eee.p4.mass - 91.1876), axis=1))
print(bestZ_idx)

Triple_eee = Triple_eee[bestZ_idx]
print(Triple_eee.lep1.pt)
print(Triple_eee.p4.mass)

leading_ele = Triple_eee.lep1
subleading_ele = Triple_eee.lep2
third_ele = Triple_eee.lep3




passing eee ch mask:  1769
[-1, 1, 1] [(0, 1, 2), (0, 2, 1)]
[1, 1, -1] [(0, 2, 1), (1, 2, 0)]
[-1, 1, 1] [(0, 1, 2), (0, 2, 1)]
[1, 1, -1] [(0, 2, 1), (1, 2, 0)]
[1, 1, -1] [(0, 2, 1), (1, 2, 0)]
passing ossf:  1755
[[51.5, 51.5], [49.4, 43.3], [62.8, 62.8], ... [240, 132], [101, 80], [35.5, 25.9]]
[[67, 131], [55.2, 61.5], [89, 69.5], ... [90.5, 83], [188, 90.2], [54.3, 59.7]]
[[0], [1], [0], [0], [0], [0], [1], [0], ... [0], [1], [0], [1], [0], [0], [1], [1]]
[[51.5], [43.3], [62.8], [60], [167], ... [93.1], [73.9], [240], [80], [25.9]]
[[67], [61.5], [89], [72.5], [81.8], ... [91.6], [70.1], [90.5], [90.2], [59.7]]


### Baseline

In [7]:
diele = Triple_eee.p4
Mee_cut_mask = ak.firsts(diele.mass) > 4
Elept_mask = ak.firsts(
    (leading_ele.pt >= 25) & (subleading_ele.pt >= 10) & (third_ele.pt >= 25)
)
Baseline_mask = Elept_mask & Mee_cut_mask
Triple_eee_base = Triple_eee[Baseline_mask]
Electron_base = Electron[Baseline_mask]
MET_base = MET[Baseline_mask] 
events_base = events[Baseline_mask]
if ak.sum(ak.num(Muon)) != 0:
    Muon_base = Muon[Baseline_mask]

if ak.sum(ak.num(Jet)) != 0:
    Jet_base = Jet[Baseline_mask]
    
    
print("passing baseline sel: ",len(events_base))

passing baseline sel:  1364


In [8]:
# Photon Selection
isgap_mask = (abs(Photon.eta) < 1.442) | (
    (abs(Photon.eta) > 1.566) & (abs(Photon.eta) < 2.5)
)

Pixel_seed_mask = ~Photon.pixelSeed
PT_ID_mask = (Photon.pt >= 20) & (Photon.cutBased > 1)

if ak.sum(ak.num(Muon)) > 0:
    dr_mask = (ak.all(Photon.metric_table(Muon) >= 0.5, axis=-1) &
              ak.all(Photon.metric_table(Electron) >= 0.5, axis=-1))
else:
    dr_mask = ak.all(Photon.metric_table(Electron) >= 0.5, axis=-1)

genPartFlav_mask =  (Photon.genPartFlav == 1)
PhoSelmask = (genPartFlav_mask & PT_ID_mask & isgap_mask & Pixel_seed_mask & dr_mask)
Photon = Photon[PhoSelmask]

A_photon_mask = ak.num(Photon) > 0

def make_leading_pair(target, base):
    return target[ak.argmax(base.pt, axis=1, keepdims=True)]
leading_pho = make_leading_pair(Photon, Photon)

# --veto Bjet
dr_jet_ele_mask = ak.all(
    Jet.metric_table(Electron) >= 0.5, axis=-1
)
if ak.sum(ak.num(Muon)) != 0:
    dr_jet_mu_mask = ak.all(Jet.metric_table(Muon) >= 0.5, axis=-1)
    bJet_mask =  (Jet.pt > 10) & (abs(Jet.eta) <2.4) & (dr_jet_ele_mask) & (dr_jet_mu_mask) & (Jet.btagDeepB > 0.7665)

else:
    bJet_mask =  (Jet.pt > 10) & (abs(Jet.eta) <2.4) & (dr_jet_ele_mask) & (Jet.btagDeepB > 0.7665)
Jet = Jet[bJet_mask]

### Signal

In [9]:
Zmass_window_mask = ak.firsts(abs(Triple_eee.p4.mass - 91.1876)) <= 15
MET_mask = MET.pt > 20
bjet_veto = ak.num(Jet) == 0
SR_mask =  (Zmass_window_mask & MET_mask & bjet_veto)
SR_mask	= (Baseline_mask  & A_photon_mask & SR_mask)


Triple_eee_SR= Triple_eee[SR_mask]
leading_pho_SR= leading_pho[SR_mask]
events_SR= events[SR_mask]

print("passing SR: ",len(events_SR))

passing SR:  204


---
---
# eem channel

### ossf

In [6]:
# remove evt with conataining loose but not tight leptons
Loose_lepton_veto = ((ak.num(LoosebutNotTightElectron) + ak.num(LoosebutNotTightMuon)) == 0)
eem_ch_mask       = (ak.num(Electron)==2) & (ak.num(Muon)==1)

Ele_Mask = (Loose_lepton_veto & eem_ch_mask)

Electron = Electron[Ele_Mask]
Muon = Muon[Ele_Mask]
Photon = Photon[Ele_Mask]
MET = MET[Ele_Mask]
Jet = Jet[Ele_Mask]
events = events[Ele_Mask]

print("passing eem ch mask: ",len(events))


ossf_mask = (Electron.charge[:, 0] + Electron.charge[:, 1] == 0)
Electron = Electron[ossf_mask]
Muon = Muon[ossf_mask]
Photon = Photon[ossf_mask]
MET = MET[ossf_mask]
Jet = Jet[ossf_mask]
events = events[ossf_mask]

print("passing ossf mask: ", len(events))

passing eem ch mask:  2160
passing ossf mask:  2128


### Baseline

In [9]:
diele = Electron[:,0] + Electron[:,1]
Mee_cut_mask = (diele.mass) > 4
Elept_mask = (
    (Electron[:,0].pt >= 25) & (Electron[:,1].pt >= 10) & ak.firsts(Muon.pt >= 25)
)
Baseline_mask = Elept_mask & Mee_cut_mask
diele_base = diele[Baseline_mask]
Electron_base = Electron[Baseline_mask]
MET_base = MET[Baseline_mask] 
events_base = events[Baseline_mask]
if ak.sum(ak.num(Muon)) != 0:
    Muon_base = Muon[Baseline_mask]

if ak.sum(ak.num(Jet)) != 0:
    Jet_base = Jet[Baseline_mask]
    
    
print("passing baseline sel: ",len(events_base))

passing baseline sel:  1727


### Signal

In [ ]:
# Photon Selection
isgap_mask = (abs(Photon.eta) < 1.442) | (
    (abs(Photon.eta) > 1.566) & (abs(Photon.eta) < 2.5)
)

Pixel_seed_mask = ~Photon.pixelSeed
PT_ID_mask = (Photon.pt >= 20) & (Photon.cutBased > 1)

if ak.sum(ak.num(Muon)) > 0:
    dr_mask = (ak.all(Photon.metric_table(Muon) >= 0.5, axis=-1) &
              ak.all(Photon.metric_table(Electron) >= 0.5, axis=-1))
else:
    dr_mask = ak.all(Photon.metric_table(Electron) >= 0.5, axis=-1)

genPartFlav_mask =  (Photon.genPartFlav == 1)
PhoSelmask = (genPartFlav_mask & PT_ID_mask & isgap_mask & Pixel_seed_mask & dr_mask)
Photon = Photon[PhoSelmask]

A_photon_mask = ak.num(Photon) > 0

def make_leading_pair(target, base):
    return target[ak.argmax(base.pt, axis=1, keepdims=True)]
leading_pho = make_leading_pair(Photon, Photon)

# --veto Bjet
dr_jet_ele_mask = ak.all(
    Jet.metric_table(Electron) >= 0.5, axis=-1
)
if ak.sum(ak.num(Muon)) != 0:
    dr_jet_mu_mask = ak.all(Jet.metric_table(Muon) >= 0.5, axis=-1)
    bJet_mask =  (Jet.pt > 10) & (abs(Jet.eta) <2.4) & (dr_jet_ele_mask) & (dr_jet_mu_mask) & (Jet.btagDeepB > 0.7665)

else:
    bJet_mask =  (Jet.pt > 10) & (abs(Jet.eta) <2.4) & (dr_jet_ele_mask) & (Jet.btagDeepB > 0.7665)
Jet = Jet[bJet_mask]

In [11]:
Zmass_window_mask = (abs(diele.mass - 91.1876)) <= 15
MET_mask = MET.pt > 20
bjet_veto = ak.num(Jet) == 0
SR_mask =  (Zmass_window_mask & MET_mask & bjet_veto)
SR_mask	= (Baseline_mask  & A_photon_mask & SR_mask)


diele_SR= diele[SR_mask]
leading_pho_SR= leading_pho[SR_mask]
events_SR= events[SR_mask]

print("passing SR: ",len(events_SR))

passing SR:  240
